In [1]:
!pip install bs4

!pip install selenium

  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
  Using cached trio-0.29.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/9.5 MB 4.2 MB/s eta 0:00:03
   ------------ --------------------------- 2.9/9.5 MB 8.0 MB/s eta 0:00:01
   ---

In [3]:
!pip install pandas

  Using cached pandas-2.2.3-cp39-cp39-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl.metadata (59 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp39-cp39-win_amd64.whl (11.6 MB)
Using cached numpy-2.0.2-cp39-cp39-win_amd64.whl (15.9 MB)
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)


In [4]:
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# Set up Selenium WebDriver (requires ChromeDriver)
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
service = Service("/path/to/chromedriver")  # Update with your chromedriver path
driver = webdriver.Chrome(service=service, options=chrome_options)

# Function to scrape job details
def scrape_job_details(url):
    driver.get(url)
    time.sleep(3)  # Allow time for dynamic content to load
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # Extract job description text
    job_desc = soup.find("div", class_="description__text").get_text(" ", strip=True) if soup.find("div", class_="description__text") else ""
    return job_desc

# Function to extract structured information
def extract_info(job_desc):
    salary = re.findall(r"\$\d{2,3}(?:,\d{3})*(?:\s*-\s*\$\d{2,3}(?:,\d{3})*)?", job_desc)
    experience = re.findall(r"(\d+)\+?\s*(?:years?|yrs?)\s*(?:of experience)?", job_desc, re.IGNORECASE)
    tech_skills = re.findall(r"\b(Python|SQL|TensorFlow|PyTorch|Java|AWS|Hadoop|Spark)\b", job_desc, re.IGNORECASE)
    education = re.findall(r"(Bachelor's|Master's|PhD) in [A-Za-z\s]+", job_desc)

    return {
        "salary": salary[0] if salary else "Not specified",
        "experience": experience[0] + " years" if experience else "Not specified",
        "tech_skills": list(set(tech_skills)),
        "education": education[0] if education else "Not specified"
    }

# Load CSV
df = pd.read_csv("jobs.csv")

# Scrape and extract information
results = []
for index, row in df.iterrows():
    job_desc = scrape_job_details(row["job_detail_url"])
    job_info = extract_info(job_desc)
    results.append({
        "job_title": row["job_title"],
        "company_name": row["company_name"],
        "salary": job_info["salary"],
        "experience": job_info["experience"],
        "tech_skills": ", ".join(job_info["tech_skills"]),
        "education": job_info["education"]
    })

# Convert results to DataFrame and save
df_results = pd.DataFrame(results)
df_results.to_csv("jobs_with_details.csv", index=False)

# Close driver
driver.quit()

print("Extraction complete! Saved to jobs_with_details.csv")


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
